In [1]:
import torch
import torch_geometric
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm

sys.path.append('../')

from torch_geometric.contrib.nn.models import GRBCDAttack
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from engine.model.GCN import SimpleGCN

c:\Users\shyun\AppData\Local\anaconda3\envs\lig2023\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\shyun\AppData\Local\anaconda3\envs\lig2023\Lib\site-packages\torch_geometric\contrib\__init__.py:8: UserWarning: 'torch_geometric.contrib' contains experimental code and is subject to change. Please use with caution.
  warnings.warn(


In [2]:
dataName = 'Citeseer'
model = torch.load(f'../data/checkpoint/simpleGCN_{dataName}.pt').to('cuda')
attack = GRBCDAttack(model , 10, allow_unused =True).to('cuda')

In [3]:
dataset = Planetoid(f'./{dataName}', dataName, pre_transform=NormalizeFeatures())[0].to('cuda')

trainData = dataset
testData = dataset

In [4]:
minBudget = []
pred = model(dataset.x, dataset.edge_index).argmax(dim = 1)
for d in tqdm(range(len(trainData.x))):
    flag  = True
    budget = 1
    if pred[d] != dataset.y[d]:
        minBudget.append(0)
        continue
    while flag:
        pertubed, fl = attack.attack(dataset.x, dataset.edge_index, dataset.y, budget, d)
        ppred = model(dataset.x, pertubed).argmax(dim = 1)
        if pred[d] != ppred[d]:
            minBudget.append(budget)
            flag = False
        elif budget > 125:
            minBudget.append(-1)
            flag = False
        else:
            budget = budget + 5

 18%|█▊        | 605/3327 [2:09:45<9:43:47, 12.87s/it] 


KeyboardInterrupt: 

In [ ]:
import pickle
with open(f'../result/pickles/{dataName}Budget.pkl', 'wb') as f:
    pickle.dump(minBudget, f)